<a href="https://colab.research.google.com/github/ldt9/Seasonality-of-Futures-Contracts/blob/main/Seasonal_Long_%26_Short_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [33]:
!pip install yfinance
!pip install finta
!pip install statsmodels
!pip install gitly
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)
  Attempting uninstall: kaleido
    Found existing installation: kaleido 0.1.0
    Uninstalling kaleido-0.1.0:
      Successfully uninstalled kaleido-0.1.0


In [12]:
from datetime import datetime, timedelta
import sys
import os
import math
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar = USFederalHolidayCalendar())
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools
import matplotlib.dates as mpl_dates
import yfinance as yf
from finta import TA
from statsmodels.tsa.seasonal import seasonal_decompose
from gitly.colab.plot import GitlyPlotter
gitly = GitlyPlotter('github')
gitly.config_render('github')

# ***Load Historic Data from Yahoo Finance***
Note: col names need to be lowercase to be used as imputs for the finta library

In [13]:
def load_historic_data(symbol, start_date_str, today_date_str, period, interval, prepost):
  try:
    df = yf.download(symbol, start=start_date_str, end=today_date_str, period=period, interval=interval, prepost=prepost)
    # Add Ticker
    df["Symbol"] = symbol
    df['high'] = df['High']
    df['low'] = df['Low']
    df['open'] = df['Open']
    df['close'] = df['Close']
    df = pd.DataFrame(df, columns = ['open', 'low', 'high', 'close'])
    return df
  except:
    print('Error loading stock data for ' + symbol)
    return None

# ***Calculating Seasonality, Seasonal Highs & Lows, & Appending to DF***

In [14]:
def calculate_tis(df):
  # Calculate Seasonality
  season, seasonLow, seasonHigh = seasonal(df)

  # Add to Price df
  df = pd.concat([df, season], axis=1, ignore_index=False)

  return df, seasonLow, seasonHigh

# ***Calculate Seasonality of Futures Contract***

In [15]:
#return buy trend periods and sell trend periods so algorithm only enter/exit longs in buy trends and enter/exit shorts in sell trends
def seasonal(df):

  # Debug
  # display(df)

  data = df.drop(columns=['open', 'high', 'low'], axis=1)

  # Debug
  # display(data)

  decomposition = seasonal_decompose(data.close, model='multiplicative', period=52)
  seasonal = decomposition.seasonal

  # Extra Options from seasonal_decompose
  # trend    = decomposition.trend
  # residual = decomposition.resid

  # These will later be used for entry and exit points
  seasonalLow = min(seasonal)
  seasonalHigh = max(seasonal)

  return seasonal, seasonalLow, seasonalHigh


# ***Calculate Entry and Exit Signals***

Long Entry/Exit:
*   The Seasonal Low will be our long entry signal
*   The Seasonal High will be our long exit signal

Short Entry/Exit:
*   The Seasonal High will be our short entry signal
*   The Seasonal Low will be our short exit signal.



In [16]:
def calculate_signals(df1, low, high, strat=""):
  #By default if 'strat' is left blank, long and short signals will be generated

  if strat == "LongOnly":
    df1['enter_long'] = np.where(df1['seasonal'] == low, 1, 0)
    df1['exit_long'] = np.where(df1['seasonal'] == high, 1, 0)
    df1['enter_short'] = 0
    df1['exit_short'] = 0
  elif strat == "ShortOnly":
    df1['enter_long'] = 0
    df1['exit_long'] = 0
    df1['enter_short'] = np.where(df1['seasonal'] == high, 1, 0)
    df1['exit_short'] = np.where(df1['seasonal'] == low, 1, 0)
  else:
    df1['enter_long'] = np.where(df1['seasonal'] == low, 1, 0)
    df1['exit_long'] = np.where(df1['seasonal'] == high, 1, 0)
    df1['enter_short'] = np.where(df1['seasonal'] == high, 1, 0)
    df1['exit_short'] = np.where(df1['seasonal'] == low, 1, 0)

  return df1

# ***Strategy Execution***

In [17]:
def execute_strategy(df):
    close_prices = df['close'].to_numpy()
    enter_long = df['enter_long'].to_numpy()
    exit_long = df['exit_long'].to_numpy()
    enter_short = df['enter_short'].to_numpy()
    exit_short = df['exit_short'].to_numpy()
    
    last_long_entry_price = 0
    last_short_entry_price = 0
    long_entry_prices = []
    long_exit_prices = []
    short_entry_prices = []
    short_exit_prices = []
    hold_long = 0
    hold_short = 0

    for i in range(len(close_prices)):
        current_price = close_prices[i]
        
        #  Enter long 
        if hold_long == 0 and enter_long[i] == 1:
            last_long_entry_price = current_price   # We are now Long at Current Price
            long_entry_prices.append(current_price) # Record the Current entry price in the ledger
            long_exit_prices.append(np.nan)         # We are not Exiting therefore no exit price
            hold_long = 1                           # Hold long until next signal
        #  Exit long
        elif hold_long == 1 and exit_long[i] == 1:
            long_entry_prices.append(np.nan)        # We are not Entering therefore no exit price
            long_exit_prices.append(current_price)  # Record the Current exit price in the ledger
            hold_long = 0                           # Remove the Long hold
        else:
            #  Neither entry nor exit
            long_entry_prices.append(np.nan)        # We are not Entering therefore no exit price
            long_exit_prices.append(np.nan)         # We are not Exiting therefore no exit price

        #  Enter Short 
        if hold_short == 0 and enter_short[i] == 1:
            last_short_entry_price = current_price
            short_entry_prices.append(current_price)
            short_exit_prices.append(np.nan)
            hold_short = 1
        #  Exit short
        elif hold_short == 1 and exit_short[i] == 1:
            short_entry_prices.append(np.nan)
            short_exit_prices.append(current_price)
            hold_short = 0
        else:
            #  Neither entry nor exit
            short_entry_prices.append(np.nan)
            short_exit_prices.append(np.nan)

    return long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices

# ***Plot the Results with Plotly***

Use the Python Plotly library to display the closing price with entry and exit markers and below, in a separate graph, the Seasonality.

In [18]:
def plot_graph(symbol, df, long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices, seasonHigh, seasonLow):
  fig = make_subplots(rows=2, cols = 1, subplot_titles=['close', 'Seasonality'])

  # Plot close price
  fig.add_trace(go.Scatter(x=df.Date, y=df['close'], line=dict(color="blue", width=2), name='Close'), row=1, col=1)

  # Plot Seasonality
  fig.add_trace(go.Scatter(x=df.Date, y=df['seasonal'], mode='lines', line=dict(color="blue", width=2), name='Seasonality'), row=2, col=1)
  fig.add_hline(y=seasonHigh, line=dict(color="green", width=2), row=2, col=1)
  fig.add_hline(y=seasonLow, line=dict(color="red", width=2), row=2, col=1)

  # Long Markers
  fig.add_trace(go.Scatter(x=df.Date, y=long_entry_prices, mode='markers', marker_symbol="arrow-up", marker=dict(color='green',size=9), name='Enter Long'))
  fig.add_trace(go.Scatter(x=df.Date, y=long_exit_prices, mode='markers', marker_symbol="arrow-down", marker=dict(color='red', size=9),name='Exit Long'))

  #  Short markers
  # fig.add_trace(go.Scatter(x=df.Date, y=short_entry_prices, marker_symbol="arrow-down", marker=dict(color='#8eb028',size=9),mode='markers',name='Enter Short'))
  # fig.add_trace(go.Scatter(x=df.Date, y=short_exit_prices, marker_symbol="arrow-up", marker=dict(color='#7b32a8', size=9),mode='markers',name='Exit Short'))

  fig.update_layout(
      title={'text':f"{symbol} with Seasonal Entries & Exits", 'x':0.5},
      autosize=False,
      width=1000, height=800)
  fig.update_yaxes(range=[0,1000000000],secondary_y=True)
  fig.update_yaxes(visible=False, secondary_y=True) #hide range slider

  # fig.show()
  gitly.show(fig)
  #fig.write_image('plot.png', format="png")

# ***Calculate Profit Made***


In [19]:
def calculate_profit(start_investment, long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices):
  hold_long, hold_short = 0, 0
  available_funds = start_investment
  cost_long, cost_short = 0, 0
  num_stocks_long, num_stocks_short = 0, 0
  proceeds_short = 0
  profit = 0
  for i in range(len(long_entry_prices)):
      #  Go long
      current_entry_price_long = long_entry_prices[i]
      current_exit_price_long = long_exit_prices[i]
      if not math.isnan(current_entry_price_long) and hold_long == 0:
          num_stocks_long = available_funds / current_entry_price_long
          cost_long = num_stocks_long * current_entry_price_long
          hold_long = 1

      elif hold_long == 1 and not math.isnan(current_exit_price_long):
          hold_long = 0
          proceeds = num_stocks_long * current_exit_price_long
          profit += proceeds - cost_long

      #  Go short
      current_entry_price_short = short_entry_prices[i]
      current_exit_price_short = short_exit_prices[i]
      if not math.isnan(current_entry_price_short) and hold_short == 0:
          num_stocks_short = available_funds / current_entry_price_short
          proceeds_short = num_stocks_short * current_entry_price_short
          hold_short = 1
      elif hold_short == 1 and not math.isnan(current_exit_price_short):
          hold_short = 0
          cost_short = num_stocks_short * current_exit_price_short
          profit += proceeds_short - cost_short

  return math.trunc(profit)

# ***Main Simulator***

In [20]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # Don't want to update the code, hehehehehehe

portfolio = ['CL=F', 'ZS=F', 'ZC=F', 'SI=F',    # Commodities
             'KE=F', 'NG=F', 'HG=F', 'GC=F',
             'ES=F', 'NQ=F', 'YM=F', 'RTY=F',   # Indexes
             'ZN=F', 'ZB=F', 'ZF=F', 'ZT=F'     # Bonds
             ]

returns = []
diff = []

for i in range(len(portfolio)):
  # Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
  # Fetch data by interval (including intraday if period < 60 days)
  # Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
  period = '1wk'
  interval = '1wk'
  prepost = False

  today = datetime.today()
  today_date_str = today.strftime("%Y-%m-%d")
  days = timedelta(days=7120)
  start_date = today-days
  start_date_str = datetime.strftime(start_date, "%Y-%m-%d")
  df = load_historic_data(portfolio[i], start_date_str, today_date_str, period, interval, prepost)

  start_investment = 25000

  df = df.tail(400)
  df.reset_index(inplace=True)
  df, seasonLow, seasonHigh = calculate_tis(df)

  seasonDiff = seasonHigh - seasonLow

  if seasonDiff <= 0.04: # This filters out weak seasonal markets where we cannot make at least 8%
    break

  seasonDiff = str(round(seasonDiff*100)) + '%'
  # display(seasonDiff)

  df = calculate_signals(df, seasonLow, seasonHigh)

  long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices = execute_strategy(df)



  profit = calculate_profit(start_investment, long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices)
  # print(f"Total Profit: {profit}")
  
  # debug
  # display(df)
 
  returns.append(profit/100)
  diff.append(seasonDiff)

  plot_graph(portfolio[i], df, long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices, seasonHigh, seasonLow)

for i in range(len(returns)):
  print(f"{portfolio[i]} returned {returns[i]} with an average seasonal difference of {diff[i]}")

print(f"Total Portfolio Return: {sum(returns)} or {sum(returns)/start_investment*100:.2f}% of {start_investment}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
CL=F returned 174.95 with an average seasonal difference of 9%
ZS=F returned 511.06 with an average sea